In [2]:
import locale
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd

C:\Users\Bouayad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = pd.read_csv('data_cleaned.tsv', sep='\t', encoding='utf-8-sig')

In [10]:
df.tail(200)

,review_id,date,review_rating,title,text,votes,name,id,description,rating,rating_count,features
11412,801368976,2021-08-02,5,"Best location, elegant, clean and great rooms.",We spent 5 nights at the lovely Hotel Da Vinci...,1,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN
11413,799337005,2021-07-23,5,A chic & gorgeous hotel close to the best of P...,A charming and very cozy hotel in the heart of...,0,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN
11414,782810945,2021-02-13,5,The best stay,My family came and stayed the hotel and had a ...,1,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN
11415,776539091,2020-11-05,4,Close to the seine,It is a beautiful hotel located in the beautif...,2,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN
11416,764110694,2020-08-10,5,A sanctuary away from Covid,We loved this place. The hotel and all the sta...,3,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11607,681000637,2019-06-12,5,"Clean, pleasant,would stay again",The hotel is located in a perfect spot for exp...,1,Grand Hotel des Balcons,236439,Located in the heart of the Saint Germain dist...,4.5,900.0,"['roomFeatures_housekeeping', 'roomFeatures_pr..."
11608,680933194,2019-06-12,5,"Ideal, good value hotel for a short visit to P...",A good location close to Bvd St Germain. Very...,1,Grand Hotel des Balcons,236439,Located in the heart of the Saint Germain dist...,4.5,900.0,"['roomFeatures_housekeeping', 'roomFeatures_pr..."
11609,680356183,2019-06-09,4,Central city stay,"A small old fashioned Hotel, with excellent st...",1,Grand Hotel des Balcons,236439,Located in the heart of the Saint Germain dist...,4.5,900.0,"['roomFeatures_housekeeping', 'roomFeatures_pr..."
11610,678663956,2019-06-03,5,"Charming, central, and reasonably priced",I always enjoy my stay at Grand Hotel des Balc...,1,Grand Hotel des Balcons,236439,Located in the heart of the Saint Germain dist...,4.5,900.0,"['roomFeatures_housekeeping', 'roomFeatures_pr..."
